In [1]:
import wandb

api = wandb.Api()

# Try listing projects under "simonwa01"
entity_to_check = "ucph-simonwin"
projects = api.projects(entity_to_check)

print(f"Projects under '{entity_to_check}':")
for proj in projects:
    print("    •", proj.name)


Projects under 'ucph-simonwin':
    • uncategorized
    • MedicalSegmentation
    • Sweap1
    • VariableDepthNet-Sweep


In [ ]:
import wandb
import plotly.graph_objects as go

# ─────────────────────────────────────────────────────────────────────
# 1) Entity & project exactly as in W&B
# ─────────────────────────────────────────────────────────────────────
ENTITY_NAME  = "ucph-simonwin"
PROJECT_NAME = "MedicalSegmentation"
PROJECT_PATH = f"{ENTITY_NAME}/{PROJECT_NAME}"

# ─────────────────────────────────────────────────────────────────────
# 2) Initialize W&B API (assumes wandb.login() has been done already)
# ─────────────────────────────────────────────────────────────────────
api = wandb.Api()

# ─────────────────────────────────────────────────────────────────────
# 3) Fetch all runs in the project
# ─────────────────────────────────────────────────────────────────────
all_runs = api.runs(PROJECT_PATH)

# ─────────────────────────────────────────────────────────────────────
# 4) Build the Plotly figure (same filtering: peak ≥ 0.8, y-axis 0.8–1.0)
# ─────────────────────────────────────────────────────────────────────
fig = go.Figure()

for run in all_runs:
    try:
        df = run.history(keys=["epoch", "val/dice_fullres"], pandas=True)
    except Exception:
        continue
    if df is None or df.shape[0] == 0:
        continue

    df = df.dropna(subset=["epoch", "val/dice_fullres"])
    if df.shape[0] == 0:
        continue

    if df["val/dice_fullres"].max() < 0.8:
        continue

    hover_text = [f"{run.name}<br>Dice: {v:.3f}" for v in df["val/dice_fullres"]]
    fig.add_trace(go.Scatter(
        x=df["epoch"],
        y=df["val/dice_fullres"],
        mode='lines',
        name=run.name,
        hoverinfo='text',
        hovertext=hover_text,
        line=dict(width=1),
        opacity=0.6
    ))

fig.update_layout(
    title=f"Interactive Plot: Runs in {ENTITY_NAME}/{PROJECT_NAME} (peak ≥ 0.8)",
    xaxis_title="Epoch",
    yaxis_title="val/dice_fullres",
    yaxis=dict(range=[0.8, 1.0]),
    legend_title="Run Name",
    hovermode="closest",
    width=900,
    height=600
)

# ─────────────────────────────────────────────────────────────────────
# 5) Export to HTML instead of fig.show()
# ─────────────────────────────────────────────────────────────────────
html_path = "wandb_dice_plot.html"
fig.write_html(html_path, include_plotlyjs="cdn")
print(f"Saved interactive plot to '{html_path}'.\nOpen that file in your browser to view it.")


# <span style="color:#fcba03">Leaderboards</span>:

In [ ]:
import os
import re
import time
import wandb
from requests.exceptions import HTTPError

ENTITY_NAME  = "ucph-simonwin"
PROJECT_NAME = "MedicalSegmentation"
PROJECT_PATH = f"{ENTITY_NAME}/{PROJECT_NAME}"
api = wandb.Api()
all_runs = api.runs(PROJECT_PATH)

def safe_history(run, keys, attempts=3):
    for i in range(attempts):
        try:
            return run.history(keys=keys, pandas=True)
        except HTTPError:
            time.sleep(2 ** i)
    return None

def safe_files(run, attempts=3):
    for i in range(attempts):
        try:
            return run.files()
        except HTTPError:
            time.sleep(2 ** i)
    return []

# Step 1: Build dataset‐filtered leaderboard
target_names = ["ms-unet3d", "unet-aug3d", "unet3d"]
metrics = {"val/dice_fullres": {"higher_is_better": True}}

seen_datasets = set()
for run in all_runs:
    cfg = getattr(run, "config", {}) or {}
    raw_dataset = cfg.get("dataset") if isinstance(cfg.get("dataset"), str) else run.group or ""
    if raw_dataset:
        seen_datasets.add(raw_dataset)
    time.sleep(0.1)

if not seen_datasets:
    print("No runs have config['dataset'] or run.group. Exiting.")
    exit()

print("Datasets found in this project:")
for d in sorted(seen_datasets):
    print("  •", d)

chosen_dataset = input("\nType exactly the dataset name from above and press Enter: ").strip()
if chosen_dataset not in seen_datasets:
    print(f"❌ '{chosen_dataset}' not in the list. Exiting.")
    exit()

results = {m: {arch: [] for arch in target_names} for m in metrics}

for run in all_runs:
    cfg = getattr(run, "config", {}) or {}
    arch_cfg = cfg.get("architecture", {})
    if not isinstance(arch_cfg, dict):
        time.sleep(0.1)
        continue

    model_name = arch_cfg.get("name")
    if model_name not in target_names:
        time.sleep(0.1)
        continue

    raw_dataset = cfg.get("dataset") if isinstance(cfg.get("dataset"), str) else run.group or ""
    if raw_dataset != chosen_dataset:
        time.sleep(0.1)
        continue

    df = safe_history(run, list(metrics.keys()))
    if df is None or df.empty:
        time.sleep(0.1)
        continue

    for metric_name, props in metrics.items():
        if metric_name not in df.columns:
            continue
        series = df[metric_name].dropna()
        if series.empty:
            continue
        best_val = series.max() if props["higher_is_better"] else series.min()
        results[metric_name][model_name].append((run.name, best_val))

    time.sleep(0.5)

# Step 2: Scan logs to find each run’s “trained_models/.../best_model.pth” path
pattern = re.compile(r"(trained_models/.+?)/best_model\.pth")
download_dir = "wandb_logs"
os.makedirs(download_dir, exist_ok=True)

run_to_paths = {}

for run in all_runs:
    cfg = getattr(run, "config", {}) or {}
    raw_dataset = cfg.get("dataset") if isinstance(cfg.get("dataset"), str) else run.group or ""
    if raw_dataset != chosen_dataset:
        time.sleep(0.1)
        continue

    files = safe_files(run)
    log_files = [f for f in files if f.name.lower() == "output.log"]
    if not log_files:
        time.sleep(0.1)
        continue

    log_file = log_files[0]
    local_filename = f"{run.id}_output.log"
    local_path = os.path.join(download_dir, local_filename)

    # Always download fresh, overwrite if exists
    try:
        log_file.download(root=download_dir, replace=True)
    except TypeError:
        log_file.download(root=download_dir, exist_ok=True)

    # After download, W&B writes "wandb_logs/output.log". Rename to run-specific file
    downloaded = os.path.join(download_dir, "output.log")
    if os.path.exists(downloaded):
        try:
            os.replace(downloaded, local_path)
        except Exception:
            pass

    file_to_open = local_path if os.path.exists(local_path) else None
    if not file_to_open:
        time.sleep(0.1)
        continue

    try:
        with open(file_to_open, "r", encoding="utf-8", errors="ignore") as fp:
            content = fp.read()
    except Exception:
        time.sleep(0.1)
        continue

    matches = pattern.findall(content)
    if matches:
        run_to_paths[run.name] = sorted(set(matches))

    time.sleep(0.5)

# Step 3: Print aligned leaderboard with paths
# ─ Step 3: Print “aligned” leaderboard with paths ────────────────────────
for metric_name, model_dict in results.items():
    print(f"\n=== Leaderboard for metric: {metric_name}  |  Dataset: {chosen_dataset} ===\n")
    for model_name in target_names:
        run_list = model_dict.get(model_name, [])
        if not run_list:
            print(f"Architecture '{model_name}': NO runs found in '{chosen_dataset}'.\n")
            continue

        # Sort and take top 3
        reverse_sort = metrics[metric_name]["higher_is_better"]
        sorted_runs = sorted(run_list, key=lambda x: x[1], reverse=reverse_sort)[:3]

        # Find the longest run name among these three, so we can pad accordingly
        max_name_len = max(len(rn) for rn, _ in sorted_runs)

        print(f"Architecture: {model_name}")
        for rank, (run_name, val) in enumerate(sorted_runs, start=1):
            paths = run_to_paths.get(run_name, ["<no path found>"])
            path_display = "; ".join(p + "/" for p in paths)
            # Pad run_name to max_name_len, right‐align "val" in a 7-char field with 4 decimals
            print(
                f"  {rank}. "
                f"{run_name:<{max_name_len}}  "
                f"{val:>7.4f}   →  {path_display}"
            )
        print()
        


Datasets found in this project:
  • Task01_BrainTumour
  • Task04_Hippocampus

=== Leaderboard for metric: val/dice_fullres  |  Dataset: Task01_BrainTumour ===

Architecture: ms-unet3d
  1. lr_d6_w40_dropout0_a0.2_es-dice_p30      0.7264   →  trained_models/ms-unet3d/Task01_BrainTumour/2025-05-31_14:08:48_lr_d6_w40_dropout0_a0.2_es-dice_p30/
  2. lr_RePl_d6_w32_dropout0                  0.7208   →  trained_models/ms-unet3d/Task01_BrainTumour/2025-05-31_07:56:44_lr_RePl_d6_w32_dropout0/
  3. lr-RP_d6_w45_dropout0_a0.3_es-dice_p35   0.7187   →  trained_models/ms-unet3d/Task01_BrainTumour/2025-06-01_16:02:29_lr-RP_d6_w45_dropout0_a0.3_es-dice_p35/

Architecture: unet-aug3d
  1. bb-aug_d6_w30_dropout0         0.4889   →  trained_models/unet-aug3d/Task01_BrainTumour/2025-05-30_13:08:23_bb-aug_d6_w30_dropout0/
  2. bb-aug_d6_w40_dropout0_a0.25   0.3413   →  <no path found>/
  3. bb-aug_d6_w32_dropout0         0.3107   →  trained_models/unet-aug3d/Task01_BrainTumour/2025-05-30_12:21:41_bb-aug